In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT us.user_id,
       us.lt_day,
       (CASE 
            WHEN us.lt_day<=365 THEN '1' ELSE '0' 
            END) AS is_new,
        us.age,
        (CASE 
            WHEN us.gender_segment=1 THEN 'Ж' 
            WHEN us.gender_segment=0 THEN 'М'
            END) AS gender_segment,
        us.os_name,
        us.cpe_type_name,
        loc.country,
        loc.city,
        ags.title AS age_segment,
        trs.title AS traffic_segment,
        lts.title AS lifetime_segment,
        us.nps_score,
        (CASE WHEN us.nps_score=10 OR us.nps_score=9 THEN 'сторонник'
                  WHEN us.nps_score=8 OR us.nps_score=7 THEN 'нейтрален'
                  WHEN us.nps_score>=0 AND us.nps_score<=6 THEN 'критик'
                  END) AS nps_group
FROM user AS us
JOIN location AS loc ON us.location_id=loc.location_id
JOIN age_segment AS ags ON us.age_gr_id=ags.age_gr_id
JOIN traffic_segment AS trs ON us.tr_gr_id=trs.tr_gr_id
JOIN lifetime_segment AS lts ON us.lt_gr_id=lts.lt_gr_id;
"""

In [7]:
try:
    df = pd.read_sql(query, engine)
except: 
    print('Error loading data')
else: 
    print('Upload was successful')
df.head()

Upload was successful


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Ж,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонник
1,A001WF,2344,0,53.0,М,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонник
2,A003Q7,467,0,57.0,М,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонник
3,A004TB,4190,0,44.0,Ж,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонник
4,A004XT,1163,0,24.0,М,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонник


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)